<a href="https://colab.research.google.com/github/lindseyvanosky/PCA-Exercise/blob/main/LV_PCA_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PCA Exercise 
- Lindsey Vanosky 
- 07.29.22

#Import, Upload & Inspect

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, classification_report
from sklearn.neighbors import KNeighborsClassifier


# load the dataset
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
# view the shape of the dataset
mnist.data.head()


,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
mnist.target.head()

0    5
1    0
2    4
3    1
4    9
Name: class, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [ ]:
df = mnist.data
target = mnist.target

#Preprocessing

In [ ]:
x = df
y = target

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 42)


In [ ]:
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.fit_transform(y_test)

In [ ]:
scaler = StandardScaler()
pca = PCA(n_components = .95)

pca_pipe = make_pipeline(scaler, pca)
pca_pipe.fit(x_train, y_train_le)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=0.95))])

#KNN Models

###Model 1 - PCA

In [ ]:
pca_knn = KNeighborsClassifier()
pca_knn_pipe = make_pipeline(pca_pipe, pca_knn)
pca_knn_pipe.fit(x_train, y_train_le)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('standardscaler', StandardScaler()),
                                 ('pca', PCA(n_components=0.95))])),
                ('kneighborsclassifier', KNeighborsClassifier())])

###Model 2 - Original Data

In [ ]:
knn = KNeighborsClassifier()
knn_pipe = make_pipeline(scaler, knn)
knn_pipe.fit(x_train, y_train_le)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsclassifier', KNeighborsClassifier())])

#Predictions

###Model 1 - PCA

In [ ]:
%%time
pca_pred = pca_knn_pipe.predict(x_test)

CPU times: user 56.1 s, sys: 1.3 s, total: 57.4 s
Wall time: 38.3 s


###Model 2 - Original Data

In [ ]:
%%time
org_pred = knn_pipe.predict(x_test)

CPU times: user 1min 48s, sys: 3.32 s, total: 1min 52s
Wall time: 1min 10s


#Performance Metrics

##Model 1 - PCA

In [ ]:
#Recall Score 
pca_classification = classification_report(y_test_le, pca_pred)
print(pca_classification)

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1714
           1       0.96      0.99      0.97      1977
           2       0.95      0.94      0.94      1761
           3       0.94      0.94      0.94      1806
           4       0.94      0.94      0.94      1587
           5       0.95      0.93      0.94      1607
           6       0.96      0.98      0.97      1761
           7       0.94      0.93      0.94      1878
           8       0.97      0.90      0.93      1657
           9       0.91      0.93      0.92      1752

    accuracy                           0.95     17500
   macro avg       0.95      0.95      0.95     17500
weighted avg       0.95      0.95      0.95     17500



##Model 2 - Original Data

In [ ]:
org_classification = classification_report(y_test_le, org_pred)
print(org_classification)

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1714
           1       0.95      0.99      0.97      1977
           2       0.95      0.93      0.94      1761
           3       0.93      0.94      0.94      1806
           4       0.94      0.93      0.94      1587
           5       0.94      0.93      0.94      1607
           6       0.96      0.97      0.97      1761
           7       0.94      0.93      0.93      1878
           8       0.97      0.89      0.93      1657
           9       0.90      0.92      0.91      1752

    accuracy                           0.94     17500
   macro avg       0.94      0.94      0.94     17500
weighted avg       0.94      0.94      0.94     17500



#Conclusion

**1. Which model performed the best on the test set?**
<br></br>
Based off of the classisification report, the PCA model did only slightly better - a .01 difference across the board.

**2. Which model was the fastest at making predictions?**
<br></br>
The PCA model was much faster!: 
- CPU times: user 56.1 s, sys: 1.3 s, total: 57.4 s
Wall time: 38.3 s
<br></br>
The original data performed at these speeds:
-CPU times: user 1min 48s, sys: 3.32 s, total: 1min 52s
Wall time: 1min 10s
